In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()

In [ ]:
import requests
from pydantic import BaseModel

class RAG_from_scratch:
    @instrument
    def query(self, query: str) -> str:
        res = requests.post("http://localhost:8000/process_query_async", json={
            "query": query
        })
        res_data = res.json()
        return res_data["body"]

rag = RAG_from_scratch()

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI

import numpy as np

provider = OpenAI(model_engine="gpt-4-0125-preview")

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.query.args.query)
    .on_output()
)

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG v1',
    feedbacks = [f_answer_relevance])

In [ ]:
from tqdm.notebook import tqdm

test_questions = [
    "What is a liquidity pool and how can banks deceive investors through cryptocurrency volatility?",
    "Which altcoin seems to have the least correlation with Bitcoin's movements? Analyze its growth",
    "What has been the best-performing index or mutual fund over the last 10 years?",
    "I want to build a portfolio for this month. What stocks are predicted to grow according to experts?",
    "Who is Michael Burry?",
    "hello!",
    "what are the latest news regarding AAPL?",
    "I want to invest in the healthcare sector. Please provide an overview of the sector and some of the most interesting companies",
    "Who are you?",
    "What are the price targets for TSLA according to experts?"
]

with tru_rag as recording:
    for query in tqdm(test_questions):
        rag.query(query)

In [ ]:
tru.get_leaderboard(app_ids=["RAG v1"])

In [ ]:
tru.run_dashboard()

In [ ]:
tru.stop_dashboard()